### 1) Labeling using genre
Label each paragraph / element by checking it throughly using 4 label (genre), which are:
1. Announcement (label_id=1)
2. Prayer (label_id=2)
3. Sermon (label_id=3)
4. Worship (label_id=4)

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Load CSV paragraph 
df_data_paragraph = pd.read_csv('labeled_paskah_final_bastian.csv')

# Load CSV other
df_data_table = pd.read_csv('material_book_1_other.csv')
df_data_paragraph.head()

In [ ]:
df_data_paragraph.drop(columns="Unnamed: 0", inplace=True)

In [ ]:
df_data_paragraph.head()

In [ ]:
# Add new columns 'label' dan 'label_id' for labeling
df_data_paragraph['label_id'] = 0
df_data_paragraph['label'] = None
df_data_paragraph.head()

### Sermon Labeling (automatic)
Use patterns to distinguish between announcement & Sermon
- Sermon always started with the title "Khotbah" in documents
- Ended with terms '[AWi]' clue: use regex

In [ ]:
# Start Pattern for Sermon Label
import re

marker_start_shermon = df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("KHOTBAH")]
marker_end_shermon = df_data_paragraph.loc[df_data_paragraph.VALUE.apply(lambda x: bool(re.match("^\[\w{1,3}\s*\w{1,3}]$", x)))]

In [ ]:
lst_sermon_idx = []
for idx_sermon_start in marker_start_shermon.index:
    lst_start_end_idx = []
    lst_start_end_idx.append(idx_sermon_start)
    for idx_sermon_end in marker_end_shermon.index:
        if idx_sermon_end > idx_sermon_start:
            lst_start_end_idx.append(idx_sermon_end)
            break
    lst_sermon_idx.append(lst_start_end_idx)
lst_sermon_idx

In [ ]:
# Label "Khotbah .." As Sermon based on list of index that contains of khotbah content
for idx in lst_sermon_idx:
    for i in range(idx[0],idx[1]):
        df_data_paragraph.loc[(df_data_paragraph.index == i), 'label_id'] = 3
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 3)]

In [ ]:
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 3), 'label'] = 'Sermon'
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 3)]

### Worship Labeling
Semi automatic using format text style and format

In [ ]:
df_data_paragraph.groupby(['style_left', 'style_hanging']).size().nlargest(20)

In [ ]:
# Check format pattern for worship
# 993 - 426
# 426 - 426
worship_lyric = 'jangan sesat dengar sabda-Nya'
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains(worship_lyric)]

In [ ]:
df_data_paragraph.loc[(((df_data_paragraph['style_hanging'] == '426')) &
                       ((df_data_paragraph['style_left'] == '426') | (df_data_paragraph['style_left'] == '993'))), 
                     'label_id'] = 4
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 4), 'label'] = 'Worship'

In [ ]:
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 4)]

### Checking & Labeling each rows
Checking manualy using indexes

In [ ]:
search_text = 'Apakah relevansi nilai kemanusiaan baru Yesus bagi sikap tindakan Gereja di tengah krisis pandemik'
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains(search_text)]

In [ ]:
df_data_paragraph.loc[(df_data_paragraph.index) == 3032]

### Announcement Checking & Labeling
With Label_id = 1 and label = Announcement
*note: any unlabeled is converted to Announcement

In [ ]:
df_data_paragraph.loc[(df_data_paragraph.label_id == 0), 'label_id'] = 1

In [ ]:
df_data_paragraph.loc[(df_data_paragraph.label_id == 1), 'label'] = 'Announcement'

### Prayer Checking & Labeling
With Label_id = 2 and label = Prayer

In [ ]:
# Bahan Liturgi
# 689 - 696 = Rabu Abu <Votum & Salam?> <Hal 132>
# 709 - 711 = Rabu Abu <Pengakuan Dosa> <Hal 133>
# 801 - 806 = Minggu Pra Paska I <Votum & Salam?> <Hal 140>
# 817 - 819 = Minggu Pra Paska I <Pengakuan Dosa> <Hal 141>
# 822 - 823 = Minggu Pra Paska I <Pengakuan Dosa> <Hal 142>
# 825 - 826 = Minggu Pra Paska I <Pengakuan Dosa> <Hal 142>
# 878 - 879 = Minggu Pra Paska I <Berkat> <Hal 146>
# 891 - 896 = Minggu Pra Paska II <Votum & Salam?> <Hal 148>
# 903 - 904 = Minggu Pra Paska II <Pengakuan Dosa> <Hal 149>
# 962 - 964 = Minggu Pra Paska II <Berkat> <Hal 152>
# 983 - 988 = Minggu Pra Paska III <Votum & Salam?> <Hal 154>
# 999 - 1001 = Minggu Pra Paska III <Pengakuan Dosa> <Hal 155>
# 1066 - 1067 = Minggu Pra Paska III <Doa Persembahan> <Hal 159>
# 1086 - 1088 = Minggu Pra Paska III <Berkat> <Hal 160-161>
# 1112 - 1117 = Minggu Pra Paska IV <Votum & Salam?> <Hal 164>
# 1126 - 1135 = Minggu Pra Paska IV <Pengakuan Dosa> <Hal 164-165>
# 1205 - 1207 = Minggu Pra Paska IV <Doa Persembahan> <Hal 168>
# 1243 - 1248 = Minggu Pra Paska V <Votum & Salam?> <Hal 172>
# 1264, 1268, 1272 = Minggu Pra Paska V <Litani Pengakuan Dosa> <Hal 173-174>
# 1334 - 1335 = Minggu Pra Paska V <Berkat> <Hal 179>


idx_start = 1334
idx_end = 1336
for idx_not_labeled in range(idx_start, idx_end):
    df_data_paragraph.loc[(df_data_paragraph.index == idx_not_labeled), 'label_id'] = 2
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 2), 'label'] = 'Prayer'

df_data_paragraph.loc[(df_data_paragraph.index == idx_start+1)]

### Sermon Checking & Labeling
With label_id = 3 and label = Sermon

In [ ]:
# Bahan Khotbah
# 87 - 116 = Pengantar <Hal. 1-5>
# 130 - 153 = Rabu Abu 17 Feb <Hal 11-18>
# 169 - 202 = Minggu I Pra Paska <Hal 19-30>
# 244 - 265 = Minggu II Pra Paska <Hal 31-38>
# 282 - 307 = Minggu III Pra Paska <Hal 39-48>
# 334 - 354 = Minggu IV Pra Paska <Hal 49-60>
# 388 - 409 = Minggu V Pra Paska <Hal 61-69>
# Bahan Anak
# 2113 - 2123 = BA 1 <Hal 231-235>
# 2136 - 2144 = BA 1 Kelas Paud <Hal 235-237>
# 2153 - 2160 = BA 1 Kelas Kecil <Hal 237-239>
# 2168 - 2176 = BA 1 Kelas Besar <Hal 239-240>
# 2194 - 2211 = BA 2 <Hal 245-247>
# 2224 - 2233 = BA 2 Kelas Paud <Hal 248-249>
# 2243 - 2253 = BA 2 Kelas Kecil <Hal 249-251>
# 2262 - 2272 = BA 2 Kelas Besar <Hal 252-253>
# 2284 - 2295 = BA 3 <Hal 257-259>
# 2309 - 2318 = BA 3 Kelas Paud <Hal 260-261>
# 2329 - 2338 = BA 3 Kelas Kecil <Hal 262-263>
# 2349 - 2357 = BA 3 Kelas Besar <Hal 264-265>
# Bahan Pemahaman Alkitab
# 2477 - 2493 = PA 1 <Hal 285-287>
# 2500 - 2515 = PA 2 <Hal 289-291>
# 2537 - 2552 = PA 3 <Hal 293-296>
# Bahan Persekutuan Doa
# 2701 - 2713 = PD 1 <Hal 316-318>
# 2740 - 2748 = PD 2 <Hal 322-324>
# 2782 - 2784 = PD 3 <Hal 328>
# 2789 - 2795 = PD 3 <Hal 328 - 329>]
# Bahan Sarasehan
# 3032 - 3086 = Sarasehan <Hal 351-369>

idx_start = 3032
idx_end = 3087
for idx_not_labeled in range(idx_start, idx_end):
    df_data_paragraph.loc[(df_data_paragraph.index == idx_not_labeled), 'label_id'] = 3
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 3), 'label'] = 'Sermon'

df_data_paragraph.loc[(df_data_paragraph.index == idx_start+1)]

### Worship Checking & Labeling
With label_id = 4 and label = Worship

In [ ]:
# Worship Labeling <Bahan Liturgi dan Khotbah>
# 383 - 385 = Bahan Khotbah (Masa Paska Minggu IV) <Hal 59>

# Bahan Liturgi
# 677 - 688 = Rabu Abu <Hal 131>
# 700 - 708 = Rabu Abu <Hal 132>
# 727 - 731 = Rabu Abu <Hal 134>
# 758 - 769 = Rabu Abu <Hal 135-136>
# 774 - 786 = Rabu Abu <Hal 136>
# 796 - 800 = Minggu Pra Paska I <Hal 140>
# 810 - 816 = Minggu Pra Paska I <Hal 141>
# 820 - 821 = Minggu Pra Paska I <Hal 142>
# 824 = Minggu Pra Paska I <Hal 142>
# 827 = Minggu Pra Paska I <Hal 142>
# 831 - 836 = Minggu Pra Paska I <Hal 143>
# 863 - 867 = Minggu Pra Paska I <Hal 144-145>
# 872 - 875 = Minggu Pra Paska I <Hal 145>
# 881 - 883 = Minggu Pra Paska I <Hal 146>
# 888 - 890 = Minggu Pra Paska II <Hal 147>
# 900 - 902 = Minggu Pra Paska II <Hal 148>
# 906 - 910 = Minggu Pra Paska II <Hal 149>
# 916 - 919 = Minggu Pra Paska II <Hal 149-150>
# 946 - 954 = Minggu Pra Paska II <Hal 151>
# 959 - 961 = Minggu Pra Paska II <Hal 152>
# 973 - 982 = Minggu Pra Paska III <Hal 154>
# 995 - 998 = Minggu Pra Paska III <Hal 155>
# 1003 - 1005 = Minggu Pra Paska III <Hal 156>
# 1014 - 1022 = Minggu Pra Paska III <Hal 156-157>
# 1057 - 1062 = Minggu Pra Paska III <Hal 158-159>
# 1073 - 1077 = Minggu Pra Paska III <Hal 160>
# 1104 - 1111 = Minggu Pra Paska IV <Hal 163>
# 1122 - 1124 = Minggu Pra Paska IV <Hal 164>
# 1137 - 1142 = Minggu Pra Paska IV <Hal 165>
# 1151 - 1153 = Minggu Pra Paska IV <Hal 165-166>
# 1195 - 1203 = Minggu Pra Paska IV <Hal 167>
# 1211 - 1223 = Minggu Pra Paska IV <Hal 168>
# 1233 - 1234 = Minggu Pra Paska IV <Hal 169>
# 1239 - 1242 = Minggu Pra Paska V <Hal 171-172>
# 1253 - 1257 = Minggu Pra Paska V <Hal 173>
# 1262 - 1263 = Minggu Pra Paska V Mazmur 31 <Hal 173-174>
# 1266 - 1267 = Minggu Pra Paska V Mazmur 31 <Hal 173-174>
# 1270 - 1271 = Minggu Pra Paska V Mazmur 31 <Hal 173-174>
# 1274 - 1275 = Minggu Pra Paska V Mazmur 31 <Hal 173-174>
# 1284 - 1288 = Minggu Pra Paska V <Hal 175>
# 1292 - 1293 = Minggu Pra Paska V <Hal 176>
# 1318 - 1321 = Minggu Pra Paska V <Hal 177>
# 1327 - 1331 = Minggu Pra Paska V <Hal 178>
# Bahan Persekutuan Dewasa
# 2524 - 2536 = PA 3 <Hal 293>
# Bahan Persekutuan Doa
# 2690 - 2693 = PD 1 <Hal 315>
# 2696 - 2700 = PD 1 <Hal 315>
# 2715 - 2716 = PD 1 <Hal 318>
# 2722 - 2724 = PD 1 <Hal 319>
# 2728 - 2732 = PD 2 <Hal 321>
# 2735 - 2738 = PD 2 <Hal 321>
# 2750 - 2753 = PD 2 <Hal 324>
# 2758 - 2762 = PD 2 <Hal 325>
# 2766 - 2774 = PD 3 <Hal 327>
# 2777 - 2780 = PD 3 <Hal 328>
# 2785 - 2788 = PD 3 <Hal 328>
# 2797 - 2802 = PD 3 <Hal 330>
# 2807 - 2816 = PD 4 <Hal 331>

idx_start = 2807
idx_end = 2817
for idx_not_labeled in range(idx_start, idx_end):
    df_data_paragraph.loc[(df_data_paragraph.index == idx_not_labeled), 'label_id'] = 4
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 4), 'label'] = 'Worship'

df_data_paragraph.loc[(df_data_paragraph.index == idx_start + 1)]

In [ ]:
# Temporary Save
# df_data_paragraph.to_csv('labeled_paskah_partial1.csv')

### Slicing Data (Bastian's Part)
--- Menjadi Manusia Paska 2021---
Bahan Bastian:
- Bahan Dasar	 1

Bahan Kotbah 
- Rabu Abu [17 Februari 2021]	11
- Minggu I Pra Paska [21 Februari 2021]	19
- Minggu II Pra Paska [28 Februari 2021]	31
- Minggu III Pra Paska [7 Maret 2021]	39
- Minggu IV Pra Paska [14 Maret 2021]	49
- Minggu V Pra Paska [21 Maret 2021]	61

Bahan Liturgi 
- Rabu Abu [17 Februari 2021]	131
- Minggu I Pra Paska [21 Februari 2021]	139
- Minggu II Pra Paska [28 Februari 2021]	147
- Minggu III Pra Paska [7 Maret 2021]	153
- Minggu IV Pra Paska [14 Maret 2021]	163
- Minggu V Pra Paska [21 Maret 2021]	171

Bahan Anak 
- Bahan Anak 1 	231
- Bahan Anak 2	245
- Bahan Anak 3	 257

Bahan PA Dewasa
- Bahan PA Dewasa 1 	285
- Bahan PA Dewasa 2	289
- Bahan PA Dewasa 3	 293

Bahan Persekutuan Doa 
- Bahan Persekutuan Doa 1 	315
- Bahan Persekutuan Doa 2	 321
- Bahan Persekutuan Doa 3	 327

- Bahan Sarasehan	 351

In [ ]:
search_text = 'Pertanyaan dapat ditambahi, diolah atau dikembangkan sesuai dengan situasi dan kondisi jemaat'
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains(search_text)]

In [ ]:
df_data_paragraph.loc[(df_data_paragraph.index) == 3088]

In [ ]:
df_data_paragraph.iloc[2688:1338]

In [ ]:
# Slicing dan merge tiap df
df_1 = df_data_paragraph.iloc[86:130]
df_2 = df_data_paragraph.iloc[130:426]
df_3 = df_data_paragraph.iloc[675:1338]
df_4 = df_data_paragraph.iloc[2112:2374]
df_5 = df_data_paragraph.iloc[2476:2557]
df_6 = df_data_paragraph.iloc[2688:2818]
df_7 = df_data_paragraph.iloc[3032:3089]

In [ ]:
lst_df = [df_1, df_2, df_3, df_4, df_5, df_6, df_7]
df_final_sliced = pd.concat(lst_df)

In [ ]:
df_final_sliced.to_csv('book_paskah_labeled_bastian.csv', index = True)

In [2]:
df_test = pd.read_csv('book_paskah_labeled_bastian.csv')
df_test

,Unnamed: 0,TYPE,VALUE,style_type,style_left,style_hanging,numPr_type,numPr_ilvl,numPr_id,label_id,label
0,86,text,Pengantar,[],[],[],[],[],[],1,Announcement
1,87,text,"Bagi orang Kristen, hidup dimulai dari kebangk...",CT_Ind,[],[],[],[],[],3,Sermon
2,88,text,Kristus telah melewati kematian agar manusia b...,CT_Ind,[],[],[],[],[],3,Sermon
3,89,text,Gereja adalah persekutuan kebangkitan. Para mu...,CT_Ind,[],[],[],[],[],3,Sermon
4,90,text,"Melalui tema ""Menjadi Manusia Paska"" ini, LPP ...",CT_Ind,[],[],[],[],[],3,Sermon
5,91,text,"Covid, Allah yang Kalah?",[],[],[],[],[],[],3,Sermon
6,92,text,"Saat bahan dasar ini dibuat, seluruh warga dun...",[],[],[],[],[],[],3,Sermon
7,93,text,Covid menjadikan rasa damai hilang. Hilangnya ...,CT_Ind,[],[],[],[],[],3,Sermon
8,94,text,Situasi krisis ini melahirkan pertanyaan: apa ...,CT_Ind,[],[],[],[],[],3,Sermon
9,95,text,Peristiwa salib mengajak kita menghayati peris...,CT_Ind,[],[],[],[],[],3,Sermon


In [ ]:
# df_data_paragraph2 = pd.read_csv('labeled_paskah_partial1.csv')
df_data_paragraph.loc[(df_data_paragraph.index == 881)]

In [ ]:
# Final Save
df_data_paragraph.to_csv('labeled_paskah_final_bastian.csv')

In [ ]:
df_data_paragraph_final = pd.read_csv('labeled_paskah_final_bastian.csv')

In [ ]:
df_data_paragraph_final.loc[(df_data_paragraph_final.label_id == 4)]

### 2) Removing label from rows
For short and not important rows that has label (like "Pengantar", "Pembuka")

In [2]:
# read csv that has been merged (bastian & lukas)
df_paskah_paragraf = pd.read_csv("book_paskah_labeled_paragraf.csv")
df_paskah_paragraf.head()

,VALUE,label_id,label
0,Pengantar,1,Announcement
1,"Bagi orang Kristen, hidup dimulai dari kebangk...",3,Sermon
2,Kristus telah melewati kematian agar manusia b...,3,Sermon
3,Gereja adalah persekutuan kebangkitan. Para mu...,3,Sermon
4,"Melalui tema ""Menjadi Manusia Paska"" ini, LPP ...",3,Sermon


In [5]:
# Panjang string Value (text) < 15 word
list_val_unlabeled = []
for i in df_paskah_paragraf.index:
    cek_idx = df_paskah_paragraf.loc[(df_paskah_paragraf.index == i)]
    for val in cek_idx.VALUE:
        if len(val) < 15:
            list_val_unlabeled.append(i)
list_val_unlabeled

[0,
 30,
 32,
 44,
 50,
 53,
 65,
 83,
 96,
 110,
 114,
 152,
 153,
 171,
 173,
 177,
 195,
 196,
 206,
 219,
 248,
 262,
 272,
 273,
 276,
 279,
 281,
 283,
 284,
 289,
 291,
 294,
 295,
 296,
 302,
 311,
 314,
 321,
 340,
 345,
 349,
 351,
 354,
 379,
 439,
 452,
 454,
 466,
 471,
 478,
 498,
 557,
 562,
 565,
 595,
 600,
 615,
 622,
 640,
 646,
 655,
 661,
 693,
 697,
 698,
 704,
 707,
 710,
 718,
 723,
 740,
 745,
 751,
 760,
 766,
 782,
 784,
 790,
 791,
 797,
 800,
 809,
 828,
 834,
 843,
 849,
 856,
 858,
 860,
 868,
 872,
 873,
 876,
 889,
 892,
 895,
 896,
 902,
 912,
 918,
 923,
 928,
 929,
 934,
 940,
 944,
 949,
 950,
 951,
 954,
 955,
 957,
 965,
 969,
 973,
 975,
 976,
 984,
 992,
 997,
 998,
 1001,
 1002,
 1004,
 1008,
 1012,
 1017,
 1018,
 1021,
 1024,
 1025,
 1049,
 1060,
 1080,
 1081,
 1083,
 1084,
 1085,
 1088,
 1089,
 1110,
 1113,
 1116,
 1130,
 1137,
 1142,
 1149,
 1152,
 1200,
 1206,
 1215,
 1221,
 1238,
 1240,
 1243,
 1289,
 1306,
 1336,
 1355,
 1362,
 1369,
 140

In [6]:
# Change label of value with id in list_val_unlabeld to "Unlabeled" (content is to small)
for i in list_val_unlabeled:
    df_paskah_paragraf.loc[(df_paskah_paragraf.index == i), 'label_id'] = 0
    
df_paskah_paragraf.loc[(df_paskah_paragraf['label_id'] == 0), 'label'] = "Unlabeled"

In [7]:
# 329 rows (skip labeling)
df_paskah_paragraf.loc[(df_paskah_paragraf['label_id'] == 0)]

,VALUE,label_id,label
0,Pengantar,0,Unlabeled
30,Penutup,0,Unlabeled
32,Refrensi,0,Unlabeled
44,[WSN],0,Unlabeled
50,Yoel 2:12-17,0,Unlabeled
53,Mazmur 51,0,Unlabeled
65,INTI PEWARTAAN,0,Unlabeled
83,[ERY],0,Unlabeled
96,Mazmur 21:1-10,0,Unlabeled
110,Markus 1:9-15,0,Unlabeled


### Merge worship rows
One worship (song) is needed to become only one row.

In [70]:
df_paskah_paragraf.loc[(df_paskah_paragraf['label_id'] == 4)]

,VALUE,label_id,label
298,For God So Loved The World (Karena Begitu Alla...,4,Worship
299,"For God so loved the world, He gave His only S...",4,Worship
300,Karena begitu Allah cinta dunia\rHingga dib`ri...,4,Worship
537,"""Meski tak layak diriku, tetapi kar'na darah-M...",4,Worship
538,"dan kar'na kau memanggilku, 'ku datang, Yesus,...",4,Worship
539,"""Sebagaimana adanya jiwaku sungguh bercela,",4,Worship
540,"Darah-Mulah pembasuhnya; 'ku datang, Tuhan, pa...",4,Worship
583,"PKJ 13:1-2 ""KITA MASUK RUMAH-NYA""",4,Worship
584,"Kita masuk rumah-Nya,",4,Worship
585,berkumpul menyembah kepada-Nya,4,Worship


In [71]:
# Check if the index of the worship label is closed to each other by one
list_indexs_a_song = []
list_indexs_from_song = df_paskah_paragraf.loc[(df_paskah_paragraf.label_id==4)].index.to_list()

list_index_a_song = []
minx = list_indexs_from_song[0]
for index in list_indexs_from_song:
    if index==minx:
        list_index_a_song.append(index)
        continue
    else:
        if minx+1 == index:
            list_index_a_song.append(index)
            minx = index
        else:
            list_indexs_a_song.append(list_index_a_song)
            list_index_a_song=[]
            list_index_a_song.append(index)
            minx=index

In [73]:
list_indexs_a_song

[[298, 299, 300],
 [537, 538, 539, 540],
 [583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594],
 [606, 607, 608, 609, 610, 611, 612, 613, 614],
 [620, 621],
 [633, 634, 635, 636, 637],
 [664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675],
 [680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692],
 [702, 703],
 [705, 706],
 [716, 717],
 [719, 720, 721, 722],
 [726, 727],
 [730],
 [733],
 [737, 738, 739],
 [741, 742],
 [769, 770, 771, 772, 773],
 [778, 779, 780, 781],
 [787, 788, 789],
 [794, 795, 796],
 [806, 807, 808],
 [812, 813, 814, 815, 816],
 [822, 823, 824, 825],
 [852, 853, 854, 855],
 [857],
 [859],
 [865, 866, 867],
 [879, 880, 881, 882, 883, 884, 885, 886, 887, 888],
 [901],
 [903, 904],
 [909, 910, 911],
 [920, 921, 922],
 [924, 925, 926, 927],
 [963, 964],
 [966, 967, 968],
 [979, 980, 981, 982, 983],
 [999, 1000],
 [1010, 1011],
 [1013, 1014, 1015, 1016],
 [1028, 1029, 1030],
 [1043, 1044, 1045, 1046, 1047, 1048],
 [1057, 1058, 1059],
 [1065],
 [1

In [107]:
# Join 1 song for 1 list
lst_value_merge = []
for lst_idx in list_indexs_a_song:
    lst_value = []
    for idx in lst_idx:
        df_merged_worship = df_paskah_paragraf.loc[(df_paskah_paragraf.index == idx)]
        value = df_merged_worship['VALUE'].values[0]
        value_join = " ".join(value)
        lst_value.append(value)
    lst_value_merge.append(lst_value)
        

In [123]:
lst_value_merge

[['For God So Loved The World (Karena Begitu Allah Cinta Dunia)',
  'For God so loved the world, He gave His only Son\rTo die on Calv`rys tree from sin to set me free\rSomeday He`s coming back what glory that will be\rWonderful His love to me',
  'Karena begitu Allah cinta dunia\rHingga dib`rikanNya AnakNya yang tunggal\rSupaya yang percaya jangan binasa\rM`lainkan dapat hidup kekal.'],
 ['"Meski tak layak diriku, tetapi kar\'na darah-Mu...',
  "dan kar'na kau memanggilku, 'ku datang, Yesus, pada-Mu…",
  '"Sebagaimana adanya jiwaku sungguh bercela,',
  "Darah-Mulah pembasuhnya; 'ku datang, Tuhan, pada-Mu …"],
 ['PKJ 13:1-2 "KITA MASUK RUMAH-NYA"',
  'Kita masuk rumah-Nya,',
  'berkumpul menyembah kepada-Nya',
  'Kita masuk rumah-Nya,',
  'berkumpul menyembah kepada-Nya',
  'Kita masuk rumah-Nya,',
  'berkumpul menyembah kepada Kristus',
  'Menyembah Kristus Tuhan.',
  '2)\tLupakanlah dirimu, arahkan hatimu kepada-Nya',
  'Lupakanlah dirimu, arahkan hatimu kepada-Nya',
  'Lupakanlah dir

In [125]:
# Join one song into one same list ".join"
count = 0
lst_join = []
for val in range(0, len(lst_value_merge)):
    val = " ".join(lst_value_merge[val])
    lst_join.append(val)

In [133]:
lst_join[0]

'For God So Loved The World (Karena Begitu Allah Cinta Dunia) For God so loved the world, He gave His only Son\rTo die on Calv`rys tree from sin to set me free\rSomeday He`s coming back what glory that will be\rWonderful His love to me Karena begitu Allah cinta dunia\rHingga dib`rikanNya AnakNya yang tunggal\rSupaya yang percaya jangan binasa\rM`lainkan dapat hidup kekal.'

In [142]:
# Make new column "Join_Result", which is the merged song for the row
df_paskah_worship = df_paskah_paragraf.loc[(df_paskah_paragraf['label_id'] == 4)]
count = 0
lst_df_joined = []
for lst_idx in list_indexs_a_song:
    for idx in lst_idx:
        df_merged_worship = df_paskah_worship.loc[(df_paskah_worship.index == idx)]
        join_result = lst_join[count]
        df_merged_worship["Join_Result"] = join_result
        lst_df_joined.append(df_merged_worship)
    count+=1

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

...

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [143]:
lst_df_joined

[                                                 VALUE  label_id    label  \
 298  For God So Loved The World (Karena Begitu Alla...         4  Worship   
 
                                            Join_Result  
 298  For God So Loved The World (Karena Begitu Alla...  ,
                                                  VALUE  label_id    label  \
 299  For God so loved the world, He gave His only S...         4  Worship   
 
                                            Join_Result  
 299  For God So Loved The World (Karena Begitu Alla...  ,
                                                  VALUE  label_id    label  \
 300  Karena begitu Allah cinta dunia\rHingga dib`ri...         4  Worship   
 
                                            Join_Result  
 300  For God So Loved The World (Karena Begitu Alla...  ,
                                                  VALUE  label_id    label  \
 537  "Meski tak layak diriku, tetapi kar'na darah-M...         4  Worship   
 
                 

In [144]:
df_result_join = pd.concat(lst_df_joined)
df_result_join

,VALUE,label_id,label,Join_Result
298,For God So Loved The World (Karena Begitu Alla...,4,Worship,For God So Loved The World (Karena Begitu Alla...
299,"For God so loved the world, He gave His only S...",4,Worship,For God So Loved The World (Karena Begitu Alla...
300,Karena begitu Allah cinta dunia\rHingga dib`ri...,4,Worship,For God So Loved The World (Karena Begitu Alla...
537,"""Meski tak layak diriku, tetapi kar'na darah-M...",4,Worship,"""Meski tak layak diriku, tetapi kar'na darah-M..."
538,"dan kar'na kau memanggilku, 'ku datang, Yesus,...",4,Worship,"""Meski tak layak diriku, tetapi kar'na darah-M..."
539,"""Sebagaimana adanya jiwaku sungguh bercela,",4,Worship,"""Meski tak layak diriku, tetapi kar'na darah-M..."
540,"Darah-Mulah pembasuhnya; 'ku datang, Tuhan, pa...",4,Worship,"""Meski tak layak diriku, tetapi kar'na darah-M..."
583,"PKJ 13:1-2 ""KITA MASUK RUMAH-NYA""",4,Worship,"PKJ 13:1-2 ""KITA MASUK RUMAH-NYA"" Kita masuk r..."
584,"Kita masuk rumah-Nya,",4,Worship,"PKJ 13:1-2 ""KITA MASUK RUMAH-NYA"" Kita masuk r..."
585,berkumpul menyembah kepada-Nya,4,Worship,"PKJ 13:1-2 ""KITA MASUK RUMAH-NYA"" Kita masuk r..."


In [147]:
df_result_join['Join_Result'].loc[(df_result_join.index == 584)]

584    PKJ 13:1-2 "KITA MASUK RUMAH-NYA" Kita masuk r...
Name: Join_Result, dtype: object

In [150]:
# Pick the first row and changed the column "VALUE" with column "Join_Result"
lst_worship_joined = []
for idx in list_indexs_a_song:
    df_new_row = df_result_join.loc[(df_result_join.index == idx[0])]
    lst_worship_joined.append(df_new_row)

In [151]:
lst_worship_joined

[                                                 VALUE  label_id    label  \
 298  For God So Loved The World (Karena Begitu Alla...         4  Worship   
 
                                            Join_Result  
 298  For God So Loved The World (Karena Begitu Alla...  ,
                                                  VALUE  label_id    label  \
 537  "Meski tak layak diriku, tetapi kar'na darah-M...         4  Worship   
 
                                            Join_Result  
 537  "Meski tak layak diriku, tetapi kar'na darah-M...  ,
                                  VALUE  label_id    label  \
 583  PKJ 13:1-2 "KITA MASUK RUMAH-NYA"         4  Worship   
 
                                            Join_Result  
 583  PKJ 13:1-2 "KITA MASUK RUMAH-NYA" Kita masuk r...  ,
                           VALUE  label_id    label  \
 606  KJ 4:1,5 "HAI MARI SEMBAH"         4  Worship   
 
                                            Join_Result  
 606  KJ 4:1,5 "HAI MARI SEMBAH" 1)\t

In [152]:
# Concat list to df
df_new_worship = pd.concat(lst_worship_joined)
df_new_worship

,VALUE,label_id,label,Join_Result
298,For God So Loved The World (Karena Begitu Alla...,4,Worship,For God So Loved The World (Karena Begitu Alla...
537,"""Meski tak layak diriku, tetapi kar'na darah-M...",4,Worship,"""Meski tak layak diriku, tetapi kar'na darah-M..."
583,"PKJ 13:1-2 ""KITA MASUK RUMAH-NYA""",4,Worship,"PKJ 13:1-2 ""KITA MASUK RUMAH-NYA"" Kita masuk r..."
606,"KJ 4:1,5 ""HAI MARI SEMBAH""",4,Worship,"KJ 4:1,5 ""HAI MARI SEMBAH"" 1)\tHai mari sembah..."
620,"Tuhan, kami berlumuran dosa",4,Worship,"Tuhan, kami berlumuran dosa Tuhan, sudilah amp..."
633,"KJ 40:1-2 ""AJAIB BENAR ANUGERAH""",4,Worship,"KJ 40:1-2 ""AJAIB BENAR ANUGERAH"" Ajaib benar a..."
664,"Sekarang b'ri syukur, hai hati, mulut, tangan!",4,Worship,"Sekarang b'ri syukur, hai hati, mulut, tangan!..."
680,"PKJ 179:1-2 ""KASIH PALING AGUNG""",4,Worship,"PKJ 179:1-2 ""KASIH PALING AGUNG"" Kasih paling ..."
702,"NKB 34:1-2 ""SETIAMU, TUHANKU TIADA BERTARA""",4,Worship,"NKB 34:1-2 ""SETIAMU, TUHANKU TIADA BERTARA"" 1)..."
705,"Setia-Mu Tuhanku, mengharu hatiku,\rsetiap pag...",4,Worship,"Setia-Mu Tuhanku, mengharu hatiku,\rsetiap pag..."


In [153]:
df_worship_final = df_new_worship.drop(columns="VALUE")
df_worship_final.head()

,label_id,label,Join_Result
298,4,Worship,For God So Loved The World (Karena Begitu Alla...
537,4,Worship,"""Meski tak layak diriku, tetapi kar'na darah-M..."
583,4,Worship,"PKJ 13:1-2 ""KITA MASUK RUMAH-NYA"" Kita masuk r..."
606,4,Worship,"KJ 4:1,5 ""HAI MARI SEMBAH"" 1)\tHai mari sembah..."
620,4,Worship,"Tuhan, kami berlumuran dosa Tuhan, sudilah amp..."


In [154]:
df_worship_final.rename(columns={'Join_Result':'VALUE'}, inplace=True)
df_worship_final

,label_id,label,VALUE
298,4,Worship,For God So Loved The World (Karena Begitu Alla...
537,4,Worship,"""Meski tak layak diriku, tetapi kar'na darah-M..."
583,4,Worship,"PKJ 13:1-2 ""KITA MASUK RUMAH-NYA"" Kita masuk r..."
606,4,Worship,"KJ 4:1,5 ""HAI MARI SEMBAH"" 1)\tHai mari sembah..."
620,4,Worship,"Tuhan, kami berlumuran dosa Tuhan, sudilah amp..."
633,4,Worship,"KJ 40:1-2 ""AJAIB BENAR ANUGERAH"" Ajaib benar a..."
664,4,Worship,"Sekarang b'ri syukur, hai hati, mulut, tangan!..."
680,4,Worship,"PKJ 179:1-2 ""KASIH PALING AGUNG"" Kasih paling ..."
702,4,Worship,"NKB 34:1-2 ""SETIAMU, TUHANKU TIADA BERTARA"" 1)..."
705,4,Worship,"Setia-Mu Tuhanku, mengharu hatiku,\rsetiap pag..."


In [155]:
new_cols = ['VALUE','label_id','label']
df_worship_final = df_worship_final[new_cols]

In [158]:
# Convert to independent file for each genre (label)
df_worship_final.to_csv("df_worship_joined_paskah.csv")

In [159]:
df_unlabeled_final = df_paskah_paragraf.loc[(df_paskah_paragraf['label_id'] == 0)]
df_unlabeled_final.to_csv("df_unlabeled_paskah.csv")

df_announcement_final = df_paskah_paragraf.loc[(df_paskah_paragraf['label_id'] == 1)]
df_announcement_final.to_csv("df_announcement_paskah.csv")

df_prayer_final = df_paskah_paragraf.loc[(df_paskah_paragraf['label_id'] == 2)]
df_prayer_final.to_csv("df_prayer_paskah.csv")

df_sermon_final = df_paskah_paragraf.loc[(df_paskah_paragraf['label_id'] == 3)]
df_sermon_final.to_csv("df_sermon_paskah.csv")